# T1543 Create or Modify System Process

-----------------------------------------------------------------------

## Technique Description

Adversaries may create or modify system-level processes to repeatedly execute malicious payloads as part of persistence. When operating systems boot up, they can start processes that perform background system functions. On Windows and Linux, these system processes are referred to as services.(Citation: TechNet Services) On macOS, launchd processes known as [Launch Daemon](https://attack.mitre.org/techniques/T1543/004) and [Launch Agent](https://attack.mitre.org/techniques/T1543/001) are run to finish system initialization and load user specific parameters.(Citation: AppleDocs Launch Agent Daemons) 

Adversaries may install new services, daemons, or agents that can be configured to execute at startup or a repeatable interval in order to establish persistence. Similarly, adversaries may modify existing services, daemons, or agents to achieve the same effect.  

Services, daemons, or agents may be created with administrator privileges but executed under root/SYSTEM privileges. Adversaries may leverage this functionality to create or modify system processes in order to escalate privileges.(Citation: OSX Malware Detection)  

## Technique Detection

Monitor for changes to system processes that do not correlate with known software, patch cycles, etc., including by comparing results against a trusted system baseline. New, benign system processes may be created during installation of new software. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as network connections made for Command and Control, learning details about the environment through Discovery, and Lateral Movement.  

Command-line invocation of tools capable of modifying services may be unusual, depending on how systems are typically used in a particular environment. Look for abnormal process call trees from known services and for execution of other commands that could relate to Discovery or other adversary techniques. 

Monitor for changes to files associated with system-level processes.

-----------------------------------------------------------------------

### Tactics:

  * Persistence

  * Privilege-Escalation

### Platforms:

  * Windows

  * macOS

  * Linux

### Data Sources:

  * **Windows Registry:** Windows Registry Key Creation

  * **Windows Registry:** Windows Registry Key Modification

  * **File:** File Modification

  * **Driver:** Driver Load

  * **Command:** Command Execution

  * **File:** File Creation

  * **Service:** Service Modification

  * **Process:** OS API Execution

  * **Service:** Service Creation

  * **Process:** Process Creation

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1543)

  * [Appledocs Launch Agent Daemons](https://developer.apple.com/library/content/documentation/MacOSX/Conceptual/BPSystemStartup/Chapters/CreatingLaunchdJobs.html), Apple. (n.d.). Creating Launch Daemons and Agents. Retrieved July 10, 2017.

  * [Technet Services](https://technet.microsoft.com/en-us/library/cc772408.aspx), Microsoft. (n.d.). Services. Retrieved June 7, 2016.

  * [Osx Malware Detection](https://www.synack.com/wp-content/uploads/2016/03/RSA_OSX_Malware.pdf), Patrick Wardle. (2016, February 29). Let's Play Doctor: Practical OS X Malware Detection & Analysis. Retrieved July 10, 2017.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 8 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Adversaries will create or modify system services to establish persistence.

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| APT 28 | 1, 2 | |
| APT 29 | 1, 2 | |
| Turla | 1, 2 | |

## Detection Blindspots

- Odd administator tools 
- Bad logging

## Analytical References


* https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1543.003/T1543.003.md
* https://docs.microsoft.com/en-us/previous-versions/windows/it-pro/windows-server-2008-R2-and-2008/cc772408(v=ws.11)?redirectedfrom=MSDN
* https://www.welivesecurity.com/2017/03/30/carbon-paper-peering-turlas-second-stage-backdoor/
* https://www.welivesecurity.com/wp-content/uploads/2016/10/eset-sednit-part1.pdf
* https://redcanary.com/threat-detection-report/techniques/windows-service/

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Monitor for changes to system processes that do not correlate with known software, patch cycles, etc., including by comparing results against a trusted system baseline. New, benign system processes may be created during installation of new software. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as network connections made for Command and Control, learning details about the environment through Discovery, and Lateral Movement.
- Command-line invocation of tools capable of modifying services may be unusual, depending on how systems are typically used in a particular environment. Look for abnormal process call trees from known services and for execution of other commands that could relate to Discovery or other adversary techniques.
- Monitor for changes to files associated with system-level processes.
- Suspicious program execution through services may show up as outlier processes that have not been seen before when compared against historical data. Look for abnormal process call trees from known services and for execution of other commands that could relate to Discovery or other adversary techniques. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as network connections made for Command and Control, learning details about the environment through Discovery, and Lateral Movement.
- Look for changes to service Registry entries that do not correlate with known software, patch cycles, etc. Service information is stored in the Registry at HKLM\SYSTEM\CurrentControlSet\Services. Changes to the binary path and the service startup type changed from manual or disabled to automatic, if it does not typically do so, may be suspicious. Tools such as Sysinternals Autoruns may also be used to detect system service changes that could be attempts at persistence.
- Turla and APT29 has used Carbon or CosmicDuke to create these services and while it is more likely a service would be under a different name; this could be a long-hanging fruit. (and possibly be expanded as time goes on)

#### Analytic 1

  * **Information:** 'Generally, services should not be newly installed.'

  * **Source:** 'Windows Audits'

  * **Tool:** 'Kibana'

  * **Notes:** '7000 : service failed to start

    7009 : service error

    4697/7045 : A service was installed in the system.

    Look at the “Service File Name: or “ImagePath”'

  * **Query:** ```event.code : 4697 or 7045 or 7000 or 7009```

#### Analytic 2

  * **Information:** 'Previously used service names (see Hunter Notes)'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```service.name : srservice or ipvpn or hkmsvc or javamtsup```

#### Analytic 3

  * **Information:** 'Check command line parameters'

  * **Source:** 'Windows Audits, Sysmon'

  * **Tool:** 'Kibana'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```process.name : net.exe or net1.exe or sc.exe or services.exe or wmic.exe or powershell.exe AND process.command.line : *```
  
  #### Analytic 4

  * **Information:** 'look for the services registry key being modified. A new service should appear in this key.'

  * **Source:** 'Windows Audits, Sysmon'

  * **Tool:** 'Kibana'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```registry.key : *Services* AND winlog.event_data.Details : * OR TargetObject : HKLM\System\CurrentControlSet\Services\*```

-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------
